In [1]:
import pandas as pd

output_error = pd.DataFrame()

In [2]:
input_ = pd.read_excel('./file/input.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
input_['No'].tolist()

总数量：2


[1, 2]

In [3]:
a = 0

a

0

In [4]:
crawler_status = 'error'

crawler_status

'error'

In [5]:
import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

url = input_.loc[a, 'Url']

dict_param = {}
list_param = url.split('?')[1].split('&') if '?' in url else []
for param in list_param:
    dict_param[param.split('=')[0]] = param.split('=')[1]

dict_param.pop('page', '')
dict_param['sort'] = 'alphaasc'
dict_param['limit'] = '96'

url = url.split('?')[0] + '?'
for key, value in dict_param.items():
    url += f'{key}={value}&'
    
list_url, i = [], 1
while True:
    b = 0
    while True:
        b += 1
        print(b, i)
        try:
            resp = requests.get(f'{url}page={i}',
                                headers=get_header(),
                                proxies=get_proxy(),
                                timeout=(10, 10))
        
            if resp.status_code == 200:
                break
        except KeyboardInterrupt:
            break
        except:
            continue

    # = = = = = = = = = = = = = = =

    soup = BeautifulSoup(resp.text, 'lxml')
    html = etree.HTML(str(soup))

    # = = = = = = = = = = = = = = =

    list_url += html.xpath('//h4[@class="card-title NoMargin"]/a/@href')

    # = = = = = = = = = = = = = = =

    i += 1

    # = = = = = = = = = = = = = = =

    if not html.xpath('//li[@class="pagination-item pagination-item--next"]'):
        break

resp

1 1
2 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
1 12
1 13
2 13
1 14
1 15


<Response [200]>

In [6]:
len(list_url) == 0

False

In [7]:
output = pd.DataFrame({'No': [i+1 for i in range(len(list_url))],
                       'Url': list_url})

output = output.sort_values(by=['No'],
                            ascending=[True],
                            ignore_index=True)

output.to_excel('./test_url.xlsx', index=False)

output

,No,Url
0,1,https://www.cardone.com/product/windshield-wip...
1,2,https://www.cardone.com/product/windshield-wip...
2,3,https://www.cardone.com/product/windshield-wip...
3,4,https://www.cardone.com/product/windshield-wip...
4,5,https://www.cardone.com/product/windshield-wip...
...,...,...
1396,1397,https://www.cardone.com/product/windshield-wip...
1397,1398,https://www.cardone.com/product/windshield-wip...
1398,1399,https://www.cardone.com/product/windshield-wip...
1399,1400,https://www.cardone.com/product/windshield-wip...


In [8]:
crawler_status = 'ok'

crawler_status

'ok'

In [9]:
if crawler_status == 'error':
    output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

output_error

""


In [10]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[False],
                                            ignore_index=True)
    output_error.to_excel('./test_url.xlsx', index=False)

output_error

""
